# Examples with Dataframes

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## Simulation inference

WIP - how do we get convenient Poisson data here?

In [ ]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/discrete"

	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

storage := analysis.NewStateTimeStorageFromPartitions(
	[]*simulator.PartitionConfig{{
		Name:              "poisson_data",
		Iteration:         &discrete.PoissonProcessIteration{},
		Params:            simulator.NewParams(map[string][]float64{
		    "rates": {0.002, 0.001, 0.004, 0.001},
		}),
		InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
		StateHistoryDepth: 1,
		Seed:              123,
	}},
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	1667980544.0,
)
xRef := analysis.DataRef{PartitionName: "poisson_data", IsTime: true}
yRefs := analysis.DataRef{{
	PartitionName: "poisson_data",
	ValueIndices: []int{0, 1, 2, 3},
}}
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)
scatter.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")